In [1]:
from senti_c import SentenceSentimentClassification,AspectSentimentAnalysis
import pandas as pd
import os

c:\users\niuni\onedrive\桌面\senti_c\vm4senti\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 問題
    - 使用senti_c 的文本情感分類他會自動將文本中依照"。"自動分段，並根據段落輸出不同分類
        - ex : (原文)份量很大，加麵或飯都可以兩人分著吃。喜歡炸物的粉漿，不厚重。服務人員態度也好，很適合聚餐"
            - result :  份量很大，加麵或飯都可以兩人分著吃。 -> 正面
                        喜歡炸物的粉漿，不厚重。 -> 正面
                        服務人員態度也好，很適合聚餐 -> 正面
        - ex : (原文)帶家人一起來吃，看環境還不錯。但真的夠難吃，義大利面沒什麼味道，另外一道豬肉根本咬不斷 我媽直接放棄吐出來⋯唯一可以吃的還是薯條 夠雷
            - result :  帶家人一起來吃，看環境還不錯。 -> 正面
                        但真的夠難吃，義大利面沒什麼味道，另外一道豬肉根本咬不斷 我媽直接放棄吐出來⋯唯一可以吃的還是薯條 夠雷 -> 負面

In [ ]:
restaurant_comments_folder = r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed"

def senti_analysis(data,col):
    comments = data[col].to_list()
    sentence_classifier = SentenceSentimentClassification(logging_level="detail")
    result = sentence_classifier.predict(comments,run_split=True,aggregate_strategy=False)

    return result

for root, dirs, files in os.walk(restaurant_comments_folder):
    for dir in dirs:
        print("要處理的餐廳類別",dir)
    for file_name in files:
        print("正在處理的excel檔案:",os.path.join(root,file_name))
    
        if file_name.endswith(".xlsx"):
            excel_file_path = os.path.join(root,file_name)
            file_name = os.path.basename(excel_file_path)
            
            df_comments = pd.read_excel(excel_file_path)
            df_comments['評論留言原內容處理'] = df_comments['評論留言原內容處理'].str.replace("。"," ").str.replace("!"," ").str.replace("！"," ").str.replace("～"," ").str.replace("~"," ").str.replace("?"," ").str.replace("？"," ")
            df_comments = df_comments.dropna()
            comment_senti_result = senti_analysis(df_comments,"評論留言原內容處理")
            comment_senti_result.rename(columns={'Inputs':'評論留言原內容處理'},inplace=True)
            comment_senti_result = pd.concat([df_comments,comment_senti_result['Preds']],axis=1).rename(columns={'Preds':'情感分析結果'})
            
            comment_senti_result_file_name = os.path.splitext(file_name)[0]+ '_result.xlsx'
            comment_senti_result_file_path = os.path.join(root,comment_senti_result_file_name)
            comment_senti_result.to_excel(comment_senti_result_file_path,index=False)
